# Task 4


In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gsp import *

**Loading the dataset**

For this task we use the original dataset, after having cleaning it. In fact we are now importing the dataset exported from the Task 1 notebook, after assessing and improving data quality (handling missing values and outliers).

In [2]:
df = pd.read_csv('dataset/df.csv', sep=',', index_col=0)

In [3]:
df.head()

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta,Sale_per_Qta
0,536365,2010-01-12 08:26:00,2.55,17850.0,United Kingdom,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,15.30
1,536365,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,71053,WHITE METAL LANTERN,6,20.34
2,536365,2010-01-12 08:26:00,2.75,17850.0,United Kingdom,84406B,CREAM CUPID HEARTS COAT HANGER,8,22.00
3,536365,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,20.34
4,536365,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,20.34


## Modelling sequences

Now we model each customer as a sequence of baskets.

In [4]:
baskets = df.groupby('CustomerID')['BasketID'].unique().apply(list)
baskets.head()

CustomerID
12347.0    [537626, 542237, 549222, 556201, 562032, 57351...
12348.0                             [539318, 541998, 548955]
12349.0                                             [577609]
12350.0                                             [543037]
12352.0    [544156, 545323, 546869, 547390, 567505, 56869...
Name: BasketID, dtype: object

We drop the customers having performed only one shopping session.

In [5]:
min_baskets = 2

In [6]:
customers_to_drop = baskets[baskets.apply(len) < min_baskets].index

In [7]:
baskets.drop(index=customers_to_drop, inplace=True)

We compute now a dataframe where each row presents a basket ID and the products bought during said transaction.

In [8]:
transactions = df[df.CustomerID.isin(baskets.index)].groupby('BasketID')['ProdID'].unique().apply(list)
transactions.head()

BasketID
536365       [85123A, 71053, 84406B, 84029G, 84029E, 21730]
536366                                              [22633]
536367    [22745, 22748, 22749, 22310, 84969, 22623, 217...
536368                         [22960, 22913, 22912, 22914]
536369                                              [21756]
Name: ProdID, dtype: object

We now combine the two dataframe in order to compute a list of each product bought by each customer during each of his sessions.

In [9]:
df2 = pd.DataFrame(columns=['CustomerID', 'basket_list'])
for CustomerID in baskets.index:
    customer_list = []
    for BasketID in baskets.loc[CustomerID]:
        customer_list.append(transactions.loc[BasketID])
    current_customer = pd.DataFrame(data= {'CustomerID': CustomerID, 'basket_list': [customer_list]})
    df2 = pd.concat([df2, current_customer])

C:\Users\user\AppData\Local\Temp\ipykernel_24212\2228823597.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df2 = pd.concat([df2, current_customer])


In [10]:
df2.set_index('CustomerID', inplace=True)

This is the resulting dataframe.

In [11]:
df2

,basket_list
CustomerID,
12347.0,"[[85116, 22375, 71477, 22771, 22772, 22773, 22..."
12348.0,"[[84991, 21213, 22952, 21977], [21726], [22437]]"
12352.0,"[[21380, 22064, 21232, 22646, 22779, 22654, 21..."
12356.0,"[[22138, 22062, 22066, 22131, 22195, 22937, 84..."
12358.0,"[[15060B, 22059, 37447, 15056P, 15056N, 20679,..."
...,...
18273.0,"[[79302M], [79302M]]"
18282.0,"[[23295, 22089, 21108, 21109], [22699, 22818, ..."
18283.0,"[[22356, 20726, 22384, 22386, 20717, 20718, 85..."


We now format slighty modify the shape of the dataframe so to fit the requirements for using it as an input of the apriori function.

In [12]:
dataset = []
for row in df2.values:
    dataset.append(row[0])

We count the total number of sequences and events contained within the dataset.

In [13]:
print("Number of input sequences: ", len(dataset))
print("Total number of events: ", sum([len(seq) for seq in dataset])) 

Number of input sequences:  2720
Total number of events:  17378


We compute the support representing particular percentage of the dataset population (from 5% to 50%)

In [14]:
for y in [5, 10, 15, 20, 40, 50]:
    print((len(dataset)*y) / 100.0)

136.0
272.0
408.0
544.0
1088.0
1360.0


As demonstrated by the very large presence of sequences composed of the same or similar articles (see examples in the next cell), we suspect the correlation of this phenomenon to a habitual purchase of the products in question. 

Examples of sequences with similar or identical products:
- ([['LUNCH BAG RED RETROSPOT', 'LUNCH BAG WOODLAND']], 197),
- ([['ALARM CLOCK BAKELIKE GREEN'], ['ALARM CLOCK BAKELIKE GREEN']], 154)

Possible explanations for this trend are:
- the "disposable" nature of the products in question and therefore the need for customers to repurchase them continuously over time;
- the nature of wholesalers or resellers of the customers themselves who could therefore repeatedly purchase the products in question from the commercial entity under analysis in order to resell them to any third parties.

### With time costraints

Here we make some attempts to tackle the optional task of including time constraints within the identification of general sequential patterns.

The previous structure is modified in order to associate each sequence of products with the date of the cart that includes them.

In [18]:
df['BasketDate'] = pd.to_datetime(df['BasketDate']) # converting 'BasketDate' to datetime
df['BasketDate'] = df['BasketDate'].dt.date # cutting the time from to BasketDate

In [19]:
df.BasketDate

0         2010-01-12
1         2010-01-12
2         2010-01-12
3         2010-01-12
4         2010-01-12
             ...    
541904    2011-09-12
541905    2011-09-12
541906    2011-09-12
541907    2011-09-12
541908    2011-09-12
Name: BasketDate, Length: 385516, dtype: object

In [20]:
baskets = df.groupby('CustomerID')['BasketID'].unique().apply(list)
baskets.head()

CustomerID
12347.0    [537626, 542237, 549222, 556201, 562032, 57351...
12348.0                             [539318, 541998, 548955]
12349.0                                             [577609]
12350.0                                             [543037]
12352.0    [544156, 545323, 546869, 547390, 567505, 56869...
Name: BasketID, dtype: object

In order to speed up the computation the customers with less than 5 shopping sessions are excluded from the analysis.

In [21]:
min_baskets = 5

In [22]:
customers_to_drop = baskets[baskets.apply(len) < min_baskets].index

In [23]:
baskets.drop(index=customers_to_drop, inplace=True)

In [24]:
transactions = df[df.CustomerID.isin(baskets.index)].groupby('BasketID')['ProdID'].unique().apply(list)
transactions.head()

BasketID
536365       [85123A, 71053, 84406B, 84029G, 84029E, 21730]
536366                                              [22633]
536367    [22745, 22748, 22749, 22310, 84969, 22623, 217...
536368                         [22960, 22913, 22912, 22914]
536369                                              [21756]
Name: ProdID, dtype: object

In [25]:
df2 = pd.DataFrame(columns=['CustomerID', 'basket_list'])
for CustomerID in baskets.index:
    current_customer_row = []
    for BasketID in baskets.loc[CustomerID]:
        transaction_date = df[df.BasketID == BasketID].BasketDate.unique()[0]
        basket_log = [transaction_date, transactions.loc[BasketID]]
        current_customer_row.append(basket_log)
    current_customer = pd.DataFrame(data= {'CustomerID': CustomerID, 'basket_list': [current_customer_row]})
    df2 = pd.concat([df2, current_customer])

C:\Users\user\AppData\Local\Temp\ipykernel_24212\2183634504.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df2 = pd.concat([df2, current_customer])


In [26]:
df2.set_index('CustomerID', inplace=True)

This is the new structure accomodating both the sequence of products and the date of their purchase.

In [27]:
df2

,basket_list
CustomerID,
12347.0,"[[2010-07-12, [85116, 22375, 71477, 22771, 227..."
12352.0,"[[2011-02-16, [21380, 22064, 21232, 22646, 227..."
12362.0,"[[2011-02-17, [21913, 22431, 22432, 21746, 217..."
12381.0,"[[2011-12-08, [47566, 16169E, 21481, 22945, 22..."
12383.0,"[[2010-12-22, [21121, 21122, 21124, 21125, 211..."
...,...
18257.0,"[[2011-02-16, [85099B, 22957, 22502, 22961, 22..."
18260.0,"[[2010-12-16, [79321, 22452, 22453, 22454, 220..."
18272.0,"[[2011-07-04, [20754, 21563, 71459, 22557, 229..."


In [28]:
print("Number of input sequences: ", len(dataset))
print("Total number of events: ", sum([len(seq) for seq in dataset])) 

Number of input sequences:  2720
Total number of events:  17378
